In [2]:
pip install --force-reinstall numpy==1.23.5


  Using cached numpy-1.23.5-cp39-cp39-macosx_10_9_x86_64.whl.metadata (2.3 kB)
Using cached numpy-1.23.5-cp39-cp39-macosx_10_9_x86_64.whl (18.1 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --force-reinstall scipy


  Using cached scipy-1.13.1-cp39-cp39-macosx_10_9_x86_64.whl.metadata (60 kB)
  Using cached numpy-2.0.2-cp39-cp39-macosx_10_9_x86_64.whl.metadata (60 kB)
Using cached scipy-1.13.1-cp39-cp39-macosx_10_9_x86_64.whl (39.4 MB)
Using cached numpy-2.0.2-cp39-cp39-macosx_10_9_x86_64.whl (21.2 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.1
    Uninstalling scipy-1.13.1:
      Successfully uninstalled scipy-1.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awswrangler 3.9.0 requires numpy<2.0,>=1.18; python_version < "3.12", but you have numpy 2.0.2 which is incompatible.
mlflow 2.7.1 requires numpy<2, but you have numpy 2.0.2 which is incompatible.
sktime 0.15.1 requires numpy<1.25,

In [9]:
os.chdir("..")


In [8]:
%pwd

'/Users/kimiyashahamat/Desktop/DL_CV/Final Project/Sheet-Music-Parser-1/ModelTrainer'

In [10]:
import os
import random
from torch.utils.data import DataLoader


# Paths
sheet_music_folder = "data/data/muscima_split/train/sheet_music"
bbox_folder = "data/data/muscima_split/train/bounding_boxes.pkl"

# List all sheet music files
sheet_music_files = [f for f in os.listdir(sheet_music_folder) if f.endswith('.png')]

# Shuffle the files
random.seed(42)  # Set a seed for reproducibility
random.shuffle(sheet_music_files)

# Split indices for train (80%), val (10%), test (10%)
total_files = len(sheet_music_files)
train_split = int(0.8 * total_files)
val_split = int(0.9 * total_files)

# Create splits
train_files = sheet_music_files[:train_split]
val_files = sheet_music_files[train_split:val_split]
test_files = sheet_music_files[val_split:]


train_loader = DataLoader(train_files, batch_size=8, shuffle=True, num_workers=2)
val_loader = DataLoader(val_files, batch_size=8, shuffle=False, num_workers=2)
test_loader = DataLoader(test_files, batch_size=8, shuffle=False, num_workers=2)


# Print the results
print(f"Training: {len(train_files)}, Validation: {len(val_files)}, Test: {len(test_files)}")


Training: 90, Validation: 11, Test: 12


In [11]:

import torch
from torch.utils.data import Dataset
from PIL import Image
import os

class MUSCIMADataset(Dataset):
    def __init__(self, sheet_music_files, bbox_folder, sheet_music_folder, transform=None):
        self.sheet_music_files = sheet_music_files
        self.bbox_folder = bbox_folder
        self.sheet_music_folder = sheet_music_folder
        self.transform = transform

    def __len__(self):
        return len(self.sheet_music_files)

    def __getitem__(self, idx):
        # Load sheet music file
        sheet_music_file = self.sheet_music_files[idx]
        sheet_music_path = os.path.join(self.sheet_music_folder, sheet_music_file)
        sheet_image = Image.open(sheet_music_path).convert("RGB")
        
        # Get bounding box images and labels
        bbox_images = []
        bbox_labels = []
        
        for bbox_file in os.listdir(self.bbox_folder):
            if bbox_file.startswith(sheet_music_file.split(".")[0]):  # Match by sheet name
                bbox_path = os.path.join(self.bbox_folder, bbox_file)
                bbox_image = Image.open(bbox_path).convert("RGB")
                label = int(bbox_file.split("_")[-1].split(".")[0])  # Extract label from filename
                
                bbox_images.append(bbox_image)
                bbox_labels.append(label)
        
        # Apply transforms
        if self.transform:
            sheet_image = self.transform(sheet_image)
            bbox_images = [self.transform(img) for img in bbox_images]
        
        return sheet_image, bbox_images, bbox_labels

In [12]:
import torchvision
from torchvision.models.detection import FasterRCNN

# Load pre-trained Faster R-CNN model
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# Replace the classification head
num_classes = 10  # Example: number of musical symbol classes + 1 (background)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Default device:", device)
model = model.to(device)

Default device: cpu


In [14]:
for val in train_loader:
    print(val)
    break


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<string>", line 1, in <module>
  File "/Users/kimiyashahamat/anaconda3/envs/my_env/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Users/kimiyashahamat/anaconda3/envs/my_env/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
  File "/Users/kimiyashahamat/anaconda3/envs/my_env/lib/python3.9/site-packages/torch/__init__.py", line 1477, in <module>
    from .functional import *  # noqa: F403
  Fil

['CVC-MUSCIMA_W-07_N-05_D-ideal.png', 'CVC-MUSCIMA_W-01_N-14_D-ideal.png', 'CVC-MUSCIMA_W-36_N-14_D-ideal.png', 'CVC-MUSCIMA_W-01_N-10_D-ideal.png', 'CVC-MUSCIMA_W-27_N-02_D-ideal.png', 'CVC-MUSCIMA_W-16_N-06_D-ideal.png', 'CVC-MUSCIMA_W-16_N-17_D-ideal.png', 'CVC-MUSCIMA_W-42_N-08_D-ideal.png']


In [21]:
import numpy as np
print(np.__version__)

print(torch.__version__)
print(torchvision.__version__)

2.0.2
2.2.2
0.17.2


In [19]:
conda install numpy

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): conda.anaconda.org:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): conda.anaconda.org:443
/ DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/main/osx-64/current_repodata.json HTTP/1.1" 304 0
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/r/noarch/current_repodata.json HTTP/1.1" 304 0
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/r/osx-64/current_repodata.json HTTP/1.1" 304 0
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/main/noarch/current_repodata.json HTTP/1.1" 304 0
DEBUG:urllib3.connectionpool:https://conda.anaconda.org:443 "GET /t/ki-d84d7d70-2fbd-4e8b-9a

In [17]:
import torch
import torch.optim as optim
from torchvision import transforms
from PIL import Image
import numpy as np
import pickle
import os

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Transform to convert images to tensors
transform = transforms.Compose([
    transforms.ToTensor(),
])

# Function to load bounding boxes from pickle file
def load_bounding_boxes(bbox_file):
    with open(bbox_file, 'rb') as f:
        return pickle.load(f)

# Load bounding box data (assumes the data structure in the .pkl file)
bbox_data = load_bounding_boxes(bbox_folder)

# Training loop
for epoch in range(5):
    model.train()
    total_loss = 0
    
    for sheet_image_file in train_loader:
        # Since `sheet_image_file` is a list, iterate over it
        for filename in sheet_image_file:
            # Load image
            sheet_image = Image.open(os.path.join(sheet_music_folder, filename)).convert("RGB")
            sheet_image = transform(sheet_image).to(device)
            
            # Load corresponding bounding boxes and labels
            if filename in bbox_data:
                bboxes, labels = bbox_data[filename]  # Adjust depending on the data format
            
                # Prepare target dictionary for Faster R-CNN
                targets = [{
                    "boxes": torch.tensor(bboxes, dtype=torch.float32).to(device),
                    "labels": torch.tensor(labels, dtype=torch.int64).to(device)
                }]
            
                # Forward pass
                loss_dict = model([sheet_image], targets)
                losses = sum(loss for loss in loss_dict.values())
            
                # Backpropagation
                optimizer.zero_grad()
                losses.backward()
                optimizer.step()
            
                total_loss += losses.item()
    
    print(f"Epoch {epoch}, Loss: {total_loss}")



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<string>", line 1, in <module>
  File "/Users/kimiyashahamat/anaconda3/envs/my_env/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Users/kimiyashahamat/anaconda3/envs/my_env/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
  File "/Users/kimiyashahamat/anaconda3/envs/my_env/lib/python3.9/site-packages/torch/__init__.py", line 1477, in <module>
    from .functional import *  # noqa: F403
  Fil

RuntimeError: Numpy is not available

In [14]:
!conda activate my_env


usage: conda [-h] [--no-plugins] [-V] COMMAND ...
conda: error: argument COMMAND: invalid choice: 'activate' (choose from 'clean', 'compare', 'config', 'create', 'info', 'init', 'install', 'list', 'notices', 'package', 'remove', 'uninstall', 'rename', 'run', 'search', 'update', 'upgrade', 'build', 'content-trust', 'convert', 'debug', 'develop', 'doctor', 'index', 'inspect', 'metapackage', 'render', 'skeleton', 'token', 'env', 'repo', 'server', 'verify', 'pack')


In [16]:
!conda install numpy

^C

CondaError: KeyboardInterrupt



In [26]:
import torch.optim as optim

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(5):
    model.train()
    total_loss = 0
    
    for sheet_image, bbox_images, bbox_labels in train_loader:
        sheet_image = sheet_image.to(device)
        
        # Prepare target dictionary for Faster R-CNN
        targets = []
        for bboxes, labels in zip(bbox_images, bbox_labels):
            targets.append({
                "boxes": torch.tensor(bboxes, dtype=torch.float32).to(device),
                "labels": torch.tensor(labels, dtype=torch.int64).to(device)
            })
        
        # Forward pass
        loss_dict = model([sheet_image], targets)
        losses = sum(loss for loss in loss_dict.values())
        
        # Backpropagation
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        
        total_loss += losses.item()
    
    print(f"Epoch {epoch}, Loss: {total_loss}")



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<string>", line 1, in <module>
  File "/Users/kimiyashahamat/anaconda3/envs/my_env/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Users/kimiyashahamat/anaconda3/envs/my_env/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
  File "/Users/kimiyashahamat/anaconda3/envs/my_env/lib/python3.9/site-packages/torch/__init__.py", line 1477, in <module>
    from .functional import *  # noqa: F403
  Fil

ValueError: too many values to unpack (expected 3)